<a href="https://colab.research.google.com/github/DHANP2510/WEB_SCRAPPING/blob/main/WebScrappingHappinessIndex2024DataExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 19.1 MB/s eta 0:00:00


In [ ]:
"""Extracting all conturies information"""

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Setup headless browser
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)
driver.get("https://data.worldhappiness.report/table")

# Wait until render zone is present
WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CLASS_NAME, "MuiDataGrid-virtualScrollerRenderZone"))
)

all_data = []

while True:
    # Wait for data to load
    time.sleep(1)

    # Get the current visible rows
    render_zone = driver.find_element(By.CLASS_NAME, "MuiDataGrid-virtualScrollerRenderZone")
    rows = render_zone.find_elements(By.CSS_SELECTOR, 'div[data-rowindex]')

    for row in rows:
        try:
            index = row.get_attribute("data-id")
            cells = row.find_elements(By.CSS_SELECTOR, 'div.MuiDataGrid-cell.MuiDataGrid-cell--textCenter')

            ev_value = ""
            na_text = ""
            hyperlink = ""

            for cell in cells:
                field = cell.get_attribute("data-field")

                if field == "EV":
                    ev_value = cell.text.strip()
                elif field == "NA":
                    na_text = cell.text.strip()
                    try:
                        a_tag = cell.find_element(By.TAG_NAME, "a")
                        hyperlink = a_tag.get_attribute("href")
                    except:
                        hyperlink = ""

            all_data.append({
                "Index": index,
                "EV": ev_value,
                "NA": na_text,
                "Hyperlink": hyperlink
            })

        except Exception as e:
            print(f"Row error: {e}")
            continue

    # Check if "Next Page" button is disabled
    try:
        next_button = driver.find_element(By.XPATH, '//button[@title="Go to next page"]')
        if "Mui-disabled" in next_button.get_attribute("class"):
            break  # No more pages
        else:
            next_button.click()  # Go to next page
    except Exception as e:
        print("Next page button not found or error:", e)
        break

# Convert to DataFrame
Coutries_data_table1 = pd.DataFrame(all_data)

# Show result
#print(Coutries_data_table1)

# Optional: Save to CSV
# df_all.to_csv("happiness_full_data.csv", index=False)

# Close browser
driver.quit()


    Index   EV            NA                                       Hyperlink
0     778    1       Finland  https://data.worldhappiness.report/country/FIN
1     638    2       Denmark  https://data.worldhappiness.report/country/DNK
2    1118    3       Iceland  https://data.worldhappiness.report/country/ISL
3    2273    4        Sweden  https://data.worldhappiness.report/country/SWE
4    1812    5   Netherlands  https://data.worldhappiness.report/country/NLD
..    ...  ...           ...                                             ...
142  2684  143      Zimbabwe  https://data.worldhappiness.report/country/ZWE
143  1705  144        Malawi  https://data.worldhappiness.report/country/MWI
144  1355  145       Lebanon  https://data.worldhappiness.report/country/LBN
145  2169  146  Sierra Leone  https://data.worldhappiness.report/country/SLE
146    17  147   Afghanistan  https://data.worldhappiness.report/country/AFG

[147 rows x 4 columns]


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up headless Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Launch driver
driver = webdriver.Chrome(options=options)

# List to store extracted content from each country page
extracted_data = []

df1 = Coutries_data_table1

# Loop over each row in df1 to visit the hyperlink and extract content
for index, row in df1.iterrows():
    hyperlink = row['Hyperlink']
    country = row['NA']

    try:
        # Navigate to the country's hyperlink
        driver.get(hyperlink)

        # Wait until the content in class 'MuiDataGrid-virtualScrollerRenderZone' is present
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'MuiDataGrid-virtualScrollerRenderZone'))
        )

        # Locate the content inside the 'MuiDataGrid-virtualScrollerRenderZone' class
        virtual_scroller = driver.find_element(By.CLASS_NAME, 'MuiDataGrid-virtualScrollerRenderZone')

        # Extract all the rows of data within the container
        rows = virtual_scroller.find_elements(By.CSS_SELECTOR, 'div[data-id]')

        # Initialize list to store the extracted data
        country_data = []

        for row in rows:
            # Extract 'data-field' values for 'Factor' and 'Value'
            factor_element = row.find_element(By.CSS_SELECTOR, 'div[data-field="Factor"]') if len(row.find_elements(By.CSS_SELECTOR, 'div[data-field="Factor"]')) > 0 else None
            value_element = row.find_element(By.CSS_SELECTOR, 'div[data-field="Value"]') if len(row.find_elements(By.CSS_SELECTOR, 'div[data-field="Value"]')) > 0 else None

            # Get the text values if the elements exist
            factor = factor_element.text if factor_element else ''
            value = value_element.text if value_element else ''

            # Concatenate 'Factor' and 'Value' separated by a semicolon
            country_data.append(f"{factor};{value}")

        # Store the extracted data with the country name
        extracted_data.append({
            "Country": country,
            "Factor_Value": " | ".join(country_data)  # Separate multiple values with a pipe (|) for clarity
        })

    except Exception as e:
        print(f"Error processing {country}: {e}")

# Convert the extracted data into a Pandas DataFrame
Happiness_index_data_extracted = pd.DataFrame(extracted_data)

# Print the DataFrame with extracted content
#print(Happiness_index_data_extracted)

# Clean up
driver.quit()


          Country                                       Factor_Value
0         Finland  Social support;96.6% | GDP per capita;$56,017 ...
1         Denmark  Social support;96.2% | GDP per capita;$72,891 ...
2         Iceland  Social support;98.8% | GDP per capita;$65,333 ...
3          Sweden  Social support;91.7% | GDP per capita;$62,718 ...
4     Netherlands  Social support;91.4% | GDP per capita;$70,544 ...
..            ...                                                ...
142      Zimbabwe  Social support;60.9% | GDP per capita;$3,498 |...
143        Malawi  Social support;46.9% | GDP per capita;$1,638 |...
144       Lebanon  Social support;77.4% | GDP per capita;$11,398 ...
145  Sierra Leone  Social support;56.1% | GDP per capita;$3,073 |...
146   Afghanistan  Social support;- | GDP per capita;- | Healthy ...

[147 rows x 2 columns]


In [ ]:
Happiness_index_data_extracted.head()

,Country,Factor_Value
0,Finland,"Social support;96.6% | GDP per capita;$56,017 ..."
1,Denmark,"Social support;96.2% | GDP per capita;$72,891 ..."
2,Iceland,"Social support;98.8% | GDP per capita;$65,333 ..."
3,Sweden,"Social support;91.7% | GDP per capita;$62,718 ..."
4,Netherlands,"Social support;91.4% | GDP per capita;$70,544 ..."


In [ ]:
# Sample input DataFrame
df = Happiness_index_data_extracted

# Step 1: Split Factor_Value into list of "key;value"
df['Factor_List'] = df['Factor_Value'].str.split('|')

# Step 2: Parse each key-value and convert to dictionary
def parse_factor_list(factor_list):
    factor_dict = {}
    for item in factor_list:
        if ";" in item:
            key, value = item.strip().split(";", 1)
            factor_dict[key.strip()] = value.strip()
    return factor_dict

df['Factor_Dict'] = df['Factor_List'].apply(parse_factor_list)

# Step 3: Convert dict to columns
factor_df = pd.json_normalize(df['Factor_Dict'])

# Step 4: Combine with Country column
Happiness_index_data_table2 = pd.concat([df['Country'], factor_df], axis=1)

# Show final DataFrame
#print(Happiness_index_data_table2)

In [ ]:
Happiness_index_data_table2

,Country,Social support,GDP per capita,Healthy life expectancy,Freedom,Generosity,Perceptions of corruption
0,Finland,96.6%,"$56,017",-,94.6%,33.8%,18.8%
1,Denmark,96.2%,"$72,891",-,93.5%,45.9%,23.9%
2,Iceland,98.8%,"$65,333",-,96.4%,64.6%,64.9%
3,Sweden,91.7%,"$62,718",-,91.5%,53.6%,25.1%
4,Netherlands,91.4%,"$70,544",-,88.5%,60.7%,38.5%
...,...,...,...,...,...,...,...
142,Zimbabwe,60.9%,"$3,498",-,69.5%,7.7%,71.4%
143,Malawi,46.9%,"$1,638",-,75.4%,19.7%,70.3%
144,Lebanon,77.4%,"$11,398",-,50.3%,21.4%,85.5%
145,Sierra Leone,56.1%,"$3,073",-,75.1%,32.1%,85.4%


In [ ]:
Happiness_data_Intermediate = pd.merge(Coutries_data_table1, Happiness_index_data_table2, left_on="NA", right_on="Country", how="inner")
Happiness_data_Intermediate.head()

,Index,EV,NA,Hyperlink,Country,Social support,GDP per capita,Healthy life expectancy,Freedom,Generosity,Perceptions of corruption
0,778,1,Finland,https://data.worldhappiness.report/country/FIN,Finland,96.6%,"$56,017",-,94.6%,33.8%,18.8%
1,638,2,Denmark,https://data.worldhappiness.report/country/DNK,Denmark,96.2%,"$72,891",-,93.5%,45.9%,23.9%
2,1118,3,Iceland,https://data.worldhappiness.report/country/ISL,Iceland,98.8%,"$65,333",-,96.4%,64.6%,64.9%
3,2273,4,Sweden,https://data.worldhappiness.report/country/SWE,Sweden,91.7%,"$62,718",-,91.5%,53.6%,25.1%
4,1812,5,Netherlands,https://data.worldhappiness.report/country/NLD,Netherlands,91.4%,"$70,544",-,88.5%,60.7%,38.5%


In [ ]:
import pandas as pd

# URL of the Excel file
url = "https://happiness-report.s3.us-east-1.amazonaws.com/2025/Data+for+Figure+2.1+(2011%E2%80%932024).xlsx"

# Load the Excel file into a DataFrame
Happiness_index_data_table3 = pd.read_excel(url)

# Display the first few rows of the DataFrame
print(Happiness_index_data_table3[['Year', 'Rank', 'Country name', 'Ladder score']].head())


   Year  Rank Country name  Ladder score
0  2024     1      Finland         7.736
1  2023   143  Afghanistan         1.721
2  2022   137  Afghanistan         1.859
3  2021   146  Afghanistan         2.404
4  2020   150  Afghanistan         2.523


In [ ]:
Happiness_data = pd.merge(Happiness_data_Intermediate, Happiness_index_data_table3[Happiness_index_data_table3['Year'] == 2024][['Year', 'Rank', 'Country name', 'Ladder score']], left_on="NA", right_on="Country name", how="inner")
Happiness_data.head()

,Index,EV,NA,Hyperlink,Country,Social support,GDP per capita,Healthy life expectancy,Freedom,Generosity,Perceptions of corruption,Year,Rank,Country name,Ladder score
0,778,1,Finland,https://data.worldhappiness.report/country/FIN,Finland,96.6%,"$56,017",-,94.6%,33.8%,18.8%,2024,1,Finland,7.736
1,638,2,Denmark,https://data.worldhappiness.report/country/DNK,Denmark,96.2%,"$72,891",-,93.5%,45.9%,23.9%,2024,2,Denmark,7.521
2,1118,3,Iceland,https://data.worldhappiness.report/country/ISL,Iceland,98.8%,"$65,333",-,96.4%,64.6%,64.9%,2024,3,Iceland,7.515
3,2273,4,Sweden,https://data.worldhappiness.report/country/SWE,Sweden,91.7%,"$62,718",-,91.5%,53.6%,25.1%,2024,4,Sweden,7.345
4,1812,5,Netherlands,https://data.worldhappiness.report/country/NLD,Netherlands,91.4%,"$70,544",-,88.5%,60.7%,38.5%,2024,5,Netherlands,7.306


In [ ]:
Happiness_data[['Year', 'Rank', 'Country name',
       'Ladder score', 'Country', 'Social support',
       'GDP per capita', 'Healthy life expectancy', 'Freedom', 'Generosity',
       'Perceptions of corruption', 'Hyperlink']]

,Year,Rank,Country name,Ladder score,Country,Social support,GDP per capita,Healthy life expectancy,Freedom,Generosity,Perceptions of corruption,Hyperlink
0,2024,1,Finland,7.736,Finland,96.6%,"$56,017",-,94.6%,33.8%,18.8%,https://data.worldhappiness.report/country/FIN
1,2024,2,Denmark,7.521,Denmark,96.2%,"$72,891",-,93.5%,45.9%,23.9%,https://data.worldhappiness.report/country/DNK
2,2024,3,Iceland,7.515,Iceland,98.8%,"$65,333",-,96.4%,64.6%,64.9%,https://data.worldhappiness.report/country/ISL
3,2024,4,Sweden,7.345,Sweden,91.7%,"$62,718",-,91.5%,53.6%,25.1%,https://data.worldhappiness.report/country/SWE
4,2024,5,Netherlands,7.306,Netherlands,91.4%,"$70,544",-,88.5%,60.7%,38.5%,https://data.worldhappiness.report/country/NLD
...,...,...,...,...,...,...,...,...,...,...,...,...
142,2024,143,Zimbabwe,3.396,Zimbabwe,60.9%,"$3,498",-,69.5%,7.7%,71.4%,https://data.worldhappiness.report/country/ZWE
143,2024,144,Malawi,3.260,Malawi,46.9%,"$1,638",-,75.4%,19.7%,70.3%,https://data.worldhappiness.report/country/MWI
144,2024,145,Lebanon,3.188,Lebanon,77.4%,"$11,398",-,50.3%,21.4%,85.5%,https://data.worldhappiness.report/country/LBN
145,2024,146,Sierra Leone,2.998,Sierra Leone,56.1%,"$3,073",-,75.1%,32.1%,85.4%,https://data.worldhappiness.report/country/SLE
